In [1]:
import pandas as pd
import json

from vegas_dice_game.engine import GameState, Casino, Player

In [2]:
with open("InputStrategy.log") as file_:
    lines = file_.readlines()

In [3]:
def get_choice_and_game_state_from_line(line):
    choice_, game_state = line.split(' ', 1)

    game_state_json = json.loads(game_state)
    game_state = game_state_json
    game_state["casinos"] = [Casino(**c) for c in game_state["casinos"]]
    game_state["players"] = [Player(**p) for p in game_state["players"]]
    game_state = GameState(**game_state)
    
    return choice_, game_state

# get_choice_and_game_state_from_line(lines[20])

In [4]:
def calc_total_other_dice_remaining(
    game_state,
    my_id,
):
    return sum([game_state.players[id_].dice_remaining for id_ in range(len(game_state.players)) if not id_ == my_id ])

def calc_my_dice_lead(
    casino,
    my_id,
):
    my_dice = casino.placed_dice[my_id]
    max_other_dice = max([casino.placed_dice[id_] for id_ in range(len(game_state.players)) if not id_ == my_id ])
#     print(casino)
#     print(my_id)
#     print(my_dice)
#     print(max_other_dice)
    return my_dice - max_other_dice

In [5]:
def get_features_from_game_state(game_state):
    my_id = game_state.next_player_id

    features = {
        "my_dice_remaining" : game_state.players[my_id].dice_remaining,
        "total_other_dice_remaining" : calc_total_other_dice_remaining(game_state, my_id),
    }
    # print(general_features)

    for casino in game_state.casinos:
        casino_id = casino.id_
        casino_level_features = {
            f"casino_{casino_id}_num_dice_to_be_placed" : sum([1 for i in game_state.next_roll if i == casino_id]),
            f"casino_{casino_id}_my_total_dice" : casino.placed_dice[my_id],
            f"casino_{casino_id}_total_other_dice" : sum([casino.placed_dice[id_] for id_ in range(len(game_state.players)) if not id_ == my_id]),
            f"casino_{casino_id}_total_players" : sum([casino.placed_dice[id_] > 0 for id_ in range(len(game_state.players))]),
            f"casino_{casino_id}_my_dice_lead" : calc_my_dice_lead(casino, my_id),
            f"casino_{casino_id}_max_payoff" : casino.payouts[0],
            f"casino_{casino_id}_num_payoffs" : len(casino.payouts),
            f"casino_{casino_id}_total_payoff" : sum(casino.payouts),
        }
    #     print(casino_level_features)

        features = {**features, **casino_level_features}

    return features

In [6]:
rows = []
for i, line in enumerate(lines):
    choice_, game_state = get_choice_and_game_state_from_line(line)
    features = get_features_from_game_state(game_state)

    features["choice"] = choice_
    features["index_"] = i
    
    rows.append(features)

In [7]:
df = pd.DataFrame(rows)

In [8]:
df.choice.value_counts()

2    21
3    20
1    19
4    16
5    15
0    12
Name: choice, dtype: int64

In [12]:
x_vars = [
    'my_dice_remaining', 'total_other_dice_remaining',
   'casino_0_num_dice_to_be_placed', 'casino_0_my_total_dice',
   'casino_0_total_other_dice', 'casino_0_total_players',
   'casino_0_my_dice_lead', 'casino_0_max_payoff', 'casino_0_num_payoffs',
   'casino_0_total_payoff', 'casino_1_num_dice_to_be_placed',
   'casino_1_my_total_dice', 'casino_1_total_other_dice',
   'casino_1_total_players', 'casino_1_my_dice_lead',
   'casino_1_max_payoff', 'casino_1_num_payoffs', 'casino_1_total_payoff',
   'casino_2_num_dice_to_be_placed', 'casino_2_my_total_dice',
   'casino_2_total_other_dice', 'casino_2_total_players',
   'casino_2_my_dice_lead', 'casino_2_max_payoff', 'casino_2_num_payoffs',
   'casino_2_total_payoff', 'casino_3_num_dice_to_be_placed',
   'casino_3_my_total_dice', 'casino_3_total_other_dice',
   'casino_3_total_players', 'casino_3_my_dice_lead',
   'casino_3_max_payoff', 'casino_3_num_payoffs', 'casino_3_total_payoff',
   'casino_4_num_dice_to_be_placed', 'casino_4_my_total_dice',
   'casino_4_total_other_dice', 'casino_4_total_players',
   'casino_4_my_dice_lead', 'casino_4_max_payoff', 'casino_4_num_payoffs',
   'casino_4_total_payoff', 'casino_5_num_dice_to_be_placed',
   'casino_5_my_total_dice', 'casino_5_total_other_dice',
   'casino_5_total_players', 'casino_5_my_dice_lead',
   'casino_5_max_payoff', 'casino_5_num_payoffs', 'casino_5_total_payoff'
]

In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    multi_class = "multinomial"
)
model.fit(df[x_vars], df.choice)

/Users/abe/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [24]:
# model.predict_proba(df[x_vars])
y_hat = model.predict(df[x_vars])
y_hat

array(['5', '1', '1', '1', '4', '3', '4', '3', '3', '2', '0', '2', '0',
       '0', '0', '2', '2', '5', '4', '2', '2', '4', '0', '0', '1', '1',
       '4', '4', '4', '5', '5', '5', '5', '5', '0', '5', '5', '2', '3',
       '2', '5', '1', '1', '1', '1', '2', '1', '1', '1', '2', '2', '2',
       '3', '3', '2', '3', '4', '4', '4', '4', '2', '2', '5', '1', '4',
       '4', '4', '3', '1', '1', '1', '3', '1', '0', '3', '3', '0', '4',
       '4', '4', '4', '4', '3', '3', '1', '4', '4', '4', '4', '5', '1',
       '2', '5', '5', '5', '1', '2', '2', '2', '2', '3', '3', '0'],
      dtype=object)

In [ ]:
model.get_params()

In [21]:
# model.coef_

In [23]:
model.score(df[x_vars], df.choice)

0.5242718446601942

In [25]:
pd.crosstab(y_hat, df.choice)

choice,0,1,2,3,4,5
row_0,,,,,,
0,5,2,0,1,1,1
1,2,9,4,0,1,4
2,0,4,10,3,2,1
3,1,0,1,12,1,0
4,3,1,4,2,11,2
5,1,3,2,2,0,7


In [ ]:
# decision_function(X)
# Predict confidence scores for samples.
# densify()
# Convert coefficient matrix to dense array format.
# fit(X, y[, sample_weight])
# Fit the model according to the given training data.
# get_params([deep])
# Get parameters for this estimator.
# predict(X)
# Predict class labels for samples in X.
# predict_log_proba(X)
# Predict logarithm of probability estimates.
# predict_proba(X)
# Probability estimates.
# score(X, y[, sample_weight])
# Return the mean accuracy on the given test data and labels.
# set_params(**params)
# Set the parameters of this estimator.
# sparsify()

In [31]:
def get_features_from_game_state_and_casino(
    game_state,
    casino,
):
    my_id = game_state.next_player_id
    casino_id = casino.id_

    features = {
        "my_dice_remaining" : game_state.players[my_id].dice_remaining,
        "total_other_dice_remaining" : calc_total_other_dice_remaining(game_state, my_id),
        f"casino_num_dice_to_be_placed" : sum([1 for i in game_state.next_roll if i == casino_id]),
        f"casino_my_total_dice" : casino.placed_dice[my_id],
        f"casino_total_other_dice" : sum([casino.placed_dice[id_] for id_ in range(len(game_state.players)) if not id_ == my_id]),
        f"casino_total_players" : sum([casino.placed_dice[id_] > 0 for id_ in range(len(game_state.players))]),
        f"casino_my_dice_lead" : calc_my_dice_lead(casino, my_id),
        f"casino_max_payoff" : casino.payouts[0],
        f"casino_num_payoffs" : len(casino.payouts),
        f"casino_total_payoff" : sum(casino.payouts),
    }

    return features

In [51]:
rows = []
for i, line in enumerate(lines):
    choice_, game_state = get_choice_and_game_state_from_line(line)
    for casino in game_state.casinos:
        features = get_features_from_game_state_and_casino(game_state, casino)
        casino_id = casino.id_

        features["choice"] = int(choice_) == casino_id
        features["index_"] = i

        rows.append(features)

df_2 = pd.DataFrame(rows)

In [52]:
df_2.choice.value_counts()

False    515
True     103
Name: choice, dtype: int64

In [53]:
x_vars_2 = [
    'my_dice_remaining', 'total_other_dice_remaining',
    'casino_num_dice_to_be_placed', 'casino_my_total_dice',
    'casino_total_other_dice', 'casino_total_players',
    'casino_my_dice_lead', 'casino_max_payoff', 'casino_num_payoffs',
    'casino_total_payoff'
]

In [54]:
model_2 = LogisticRegression()
model_2.fit(df_2[x_vars_2], df_2.choice)

/Users/abe/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [57]:
pd.crosstab(model_2.predict(df_2[x_vars_2]), df_2.choice)

choice,False,True
row_0,,
False,509,101
True,6,2
